In [1]:
import re
from typing import List, Dict

import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.ensemble import IsolationForest
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

In [2]:
MCC_MAP = {
    "Топливо, АЗС": [5983, 5541, 5542],
    "Автомойка": [7542],

    "Одежда и обувь": [5691, 5137, 5611, 5661, 5621, 5651],

    "Такси и каршеринг": list(range(3351, 3442)) + [4121, 7512, 3990],

    "Фастфуд": [5814],

    "Электроника и техника": [5722, 5732, 5946],

    "Строительные материалы": [5039, 5198, 5211, 5231, 5251, 5713, 5714, 5712, 5074],
    "Товары для дома": [5200, 5718, 5719, 5021, 5261],

    "Автосервисы и запчасти": [7531, 7534, 7535, 7538, 5511, 5521, 5532, 5533],

    "Салоны красоты и СПА": [7230, 7297, 7298],

    "Спортивные магазины": [5655, 5940, 5941],

    "Фитнес": [7941, 7997],

    "Турагентства": [5962, 4722, 4723, 7032, 7033, 4411],

    "Отели": list(range(3501, 3990)) + list(range(3991, 4000)) + [7011],

    "Авиабилеты и ЖД билеты": [
        *list(range(3000, 3302)), 4511, 4582, 4304, 4415, 4418, 4011, 4112
    ],

    "Аптеки": [5122, 5912],

    "Медицинские услуги": [
        8011, 5976, 8031, 8042, 8043, 8049, 8050, 8062, 8021, 8071
    ],

    "Рестораны и кафе": [5811, 5812, 5813],

    "Театры, музеи и выставки": [7922, 7991, 7996, 7998, 7999],

    "Маркетплейсы": [5262, 5300, 7278],

    "Образование": [8211, 8220, 8241, 8244, 8249, 8299, 8351],

    "Детские товары": [5641, 5945],

    "Косметика": [5977],

    "Книги": [2741, 5192, 5942, 5994],

    "Цветы": [5193, 5992],

    "Кино": [7829, 7832, 7841],

    "Канцтовары": [5111, 5943],

    "Товары для животных": [742, 5995],  # 0742 -> 742

    "Бытовые услуги": [
        7210, 7211, 7216, 7217, 7251, 7349, 7379, 7622, 7623, 7629, 7641, 7692, 7699
    ],

    "Связь, интернет, ТВ": [4899, 4813, 4815, 4821, 4816, 4814],
}


In [3]:
def _find_header_row(df_raw: pd.DataFrame, required_cols: List[str], max_scan: int = 200) -> int:
    """
    Searches for the header row in messy bank Excel files.

    Args:
        df_raw: Raw DataFrame without headers.
        required_cols: Column name keywords to search for.
        max_scan: Max rows to scan.

    Returns:
        Index of header row.
    """
    best_i, best_score = 0, -1

    for i in range(min(len(df_raw), max_scan)):
        row = df_raw.iloc[i].astype(str).str.lower().tolist()

        score = 0
        for col in required_cols:
            for cell in row:
                if col.lower() in cell:
                    score += 1
                    break

        if score > best_score:
            best_score = score
            best_i = i

        if score >= len(required_cols) - 1:
            return i

    if best_score <= 0:
        raise ValueError("Не удалось найти строку заголовков.")

    return best_i


def load_bank_xlsx(path: str) -> pd.DataFrame:
    """
    Loads messy bank Excel file and extracts the real transaction table.

    Args:
        path: Path to Excel file.

    Returns:
        DataFrame with bank transactions.
    """
    df_raw = pd.read_excel(path, header=None)

    required = ["дата", "сум", "опис", "катег"]
    header_row = _find_header_row(df_raw, required_cols=required)

    df = pd.read_excel(path, header=header_row)
    df = df.dropna(how="all").reset_index(drop=True)

    df = df.loc[:, ~df.columns.astype(str).str.contains("^Unnamed")]

    return df


In [4]:
def extract_mcc(description: str) -> int | None:
    m = re.search(r"mcc[:\s]*([0-9]{4})", description.lower())
    if m:
        return int(m.group(1))
    return None


def classify_by_mcc(mcc: int) -> str | None:
    if mcc is None:
        return None

    for cat, codes in MCC_MAP.items():
        for code in codes:
            if isinstance(code, tuple):
                if code[0] <= mcc <= code[1]:
                    return cat
            else:
                if mcc == code:
                    return cat
    return None

In [5]:
def normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Converts bank columns to standard schema:
    date, description, amount, category
    """
    detected: Dict[str, list] = {
        "date": [],
        "description": [],
        "amount": [],
        "category": []
    }

    for col in df.columns:
        name = str(col).lower()

        if "дата" in name:
            detected["date"].append(col)
        elif "опис" in name or "назнач" in name:
            detected["description"].append(col)
        elif "сум" in name or "amount" in name:
            detected["amount"].append(col)
        elif "катег" in name:
            detected["category"].append(col)

    final_map = {cols[0]: key for key, cols in detected.items() if len(cols) > 0}
    df = df.rename(columns=final_map)

    if "category" not in df.columns:
        df["category"] = "Другое"

    df = df[["date", "description", "amount", "category"]]
    df["date"] = pd.to_datetime(
        df["date"],
        errors="coerce",
        dayfirst=True
    )

    df["amount"] = pd.to_numeric(df["amount"], errors="coerce")

    df = df.dropna(subset=["date", "amount"])
    df["mcc"] = df["description"].astype(str).apply(extract_mcc)

    return df


# Поиск регулярных трат

In [6]:
def filter_real_expenses(df: pd.DataFrame) -> pd.DataFrame:
    """
    Оставляем только реальные траты:
    - не переводы
    - не пополнения
    - только списания
    """
    return df[
        (~df["is_money"]) &
        (df["amount"] < 0)
        ].copy()


def normalize_description(text: str) -> str:
    """
    Removes noise from bank descriptions to make clustering work.
    """
    text = text.lower()

    # убрать цифры, номера транзакций, телефоны
    text = re.sub(r"\d+", " ", text)

    # убрать мусор
    text = re.sub(r"[^\w\s]", " ", text)

    # убрать лишние пробелы
    text = re.sub(r"\s+", " ", text).strip()

    return text


def build_merchant_id(row: pd.Series) -> str:
    """
    Формирует устойчивый идентификатор получателя платежа
    """
    base = normalize_description(row["description"])
    mcc = str(row["mcc"]) if not pd.isna(row["mcc"]) else ""
    return f"{base}|{mcc}"


def make_time_features(dates: list[pd.Timestamp], amounts: list[float]) -> np.ndarray:
    """
    Строит ML-признаки регулярности платежей
    """
    dates = sorted(dates)
    days = [d.toordinal() for d in dates]

    deltas = np.diff(days)

    return np.array([
        np.mean(deltas),
        np.std(deltas),
        np.mean(amounts),
        np.std(amounts)
    ])


def detect_recurring_payments(df: pd.DataFrame) -> pd.DataFrame:
    """
    ML-детектор регулярных платежей (подписок, сервисов, связи и т.д.)
    """
    df = filter_real_expenses(df)

    df["merchant_id"] = df.apply(build_merchant_id, axis=1)

    groups = (
        df.groupby("merchant_id")
        .agg(
            description=("description", lambda x: x.iloc[0][:60]),
            dates=("date", list),
            amounts=("amount", list),
            count=("amount", "count"),
            total=("amount", "sum")
        )
        .reset_index()
    )

    groups = groups[groups["count"] >= 3]

    if len(groups) == 0:
        return groups

    # строим ML-вектора
    features = np.vstack([
        make_time_features(row["dates"], row["amounts"])
        for _, row in groups.iterrows()
    ])

    scaler = StandardScaler()
    X = scaler.fit_transform(features)

    model = DBSCAN(eps=0.9, min_samples=1)
    labels = model.fit_predict(X)

    groups["cluster"] = labels
    groups["features"] = list(features)

    # строгий фильтр подписок
    def is_recurring(row):
        dates = pd.to_datetime(row["dates"])

        months = dates.to_period("M")

        n_months = months.nunique()
        n_payments = len(dates)

        # сколько месяцев в покрываемом интервале
        span = (months.max() - months.min()).n + 1

        coverage = n_months / span

        # стабильность суммы (но мягко)
        mean_amt = np.mean(row["amounts"])
        std_amt = np.std(row["amounts"])

        return (
                n_months >= 3 and  # минимум 3 месяца
                coverage >= 0.5 and  # платили хотя бы в половине месяцев
                abs(std_amt / mean_amt) < 0.7  # цена может расти
        )

    groups["is_recurring"] = groups.apply(is_recurring, axis=1)

    return groups[groups["is_recurring"]]


def build_recurring_features(row: pd.Series) -> np.ndarray:
    """
    Строит признаки для классификации типа регулярной траты
    """
    mean_dt, std_dt, mean_amt, std_amt = row["features"]

    return np.array([
        mean_dt,
        std_dt,
        abs(mean_amt),
        std_amt / (abs(mean_amt) + 1e-6),
        row["count"]
    ])


In [7]:
def category_statistics(df: pd.DataFrame) -> pd.DataFrame:
    total = df["amount"].abs().sum()

    stats = (
        df.groupby("final_category")["amount"]
        .sum()
        .reset_index()
        .rename(columns={"final_category": "category"})
    )

    stats["share"] = stats["amount"].abs() / total

    return stats


In [8]:
def detect_anomalies(df: pd.DataFrame) -> pd.DataFrame:
    """
    Detects anomalous transactions using Isolation Forest.

    Args:
        df: Normalized dataframe.

    Returns:
        DataFrame with an 'anomaly' column (1 = anomaly).
    """
    features = df[["amount"]].copy()

    scaler = StandardScaler()
    X = scaler.fit_transform(features)

    model = IsolationForest(contamination=0.05, random_state=42)
    df["anomaly"] = model.fit_predict(X)

    # -1 means anomaly → convert to 1
    df["anomaly"] = (df["anomaly"] == -1).astype(int)

    return df


In [9]:
def estimate_savings(df, recurring_groups, profile):
    """
    Estimates potential savings based on personal spending behavior.
    Uses user's own baseline instead of hardcoded categories.
    """
    savings = 0.0

    normal = profile[profile["is_abnormal_month"] == False]
    abnormal = profile[profile["is_abnormal_month"] == True]

    if len(normal) == 0 or len(abnormal) == 0:
        return 0.0

    baseline = normal.drop(columns=["is_abnormal_month"]).mean()

    # 1️⃣ Перерасход по месяцам
    for month, row in abnormal.iterrows():
        diff = row.drop("is_abnormal_month") - baseline

        for value in diff:
            if value > 0:
                # считаем, что 50% перерасхода можно оптимизировать
                savings += value * 0.5

    # 2️⃣ Регулярные платежи (подписки и сервисы)
    if len(recurring_groups) > 0:
        # считаем, что 60% подписок можно отключить
        savings += abs(recurring_groups["total"].sum()) * 0.6

    return round(float(savings), 2)


In [10]:
def generate_advice(
        category_stats: pd.DataFrame,
        recurring: pd.DataFrame,
        anomalies: pd.DataFrame
) -> List[str]:
    """
    Generates human-readable financial advice.

    Args:
        category_stats: Category summary.
        recurring: Recurring payments.
        anomalies: Transactions with anomalies.

    Returns:
        List of advice strings.
    """
    advice = []

    for _, row in category_stats.iterrows():
        if row["share"] > 0.35:
            advice.append(
                f"Слишком большая доля расходов на категорию '{row['category']}' ({row['share'] * 100:.1f}%)"
            )

    if len(recurring) > 0:
        advice.append("Обнаружены регулярные платежи — стоит проверить подписки и сервисы")

    if anomalies["anomaly"].sum() > 0:
        advice.append("Найдены аномально крупные траты — возможно, импульсивные покупки")

    return advice


# Расшифровка «Прочих операций»

In [11]:
from sklearn.linear_model import LogisticRegression

In [12]:
def train_category_model(df: pd.DataFrame):
    """
    Trains ML model to predict categories from descriptions.
    """
    train_df = df[df["final_category"] != "Прочие операции"]

    X = train_df["description"].astype(str)
    y = train_df["final_category"]

    vectorizer = TfidfVectorizer(max_features=1000)
    X_vec = vectorizer.fit_transform(X)

    model = LogisticRegression(max_iter=1000)
    model.fit(X_vec, y)

    return model, vectorizer


In [13]:
def classify_other_operations(df: pd.DataFrame, model, vectorizer):
    """
    Reclassifies 'Прочие операции' using ML.
    """
    mask = df["category"] == "Прочие операции"

    X = vectorizer.transform(df.loc[mask, "description"].astype(str))
    preds = model.predict(X)

    df.loc[mask, "final_category"] = preds
    df.loc[~mask, "final_category"] = df.loc[~mask, "category"]

    return df


# Строим модель привычек пользователя (ML)

In [14]:
def build_user_profile(df: pd.DataFrame):
    """
    Learns user's normal monthly spending behavior.
    """
    df["month"] = df["date"].dt.to_period("M")

    pivot = (
        df.groupby(["month", "final_category"])["amount"]
        .sum()
        .unstack(fill_value=0)
    )

    # приводим к абсолютным тратам
    pivot = pivot.abs()

    scaler = StandardScaler()
    X = scaler.fit_transform(pivot)

    model = IsolationForest(contamination=0.2, random_state=42)
    scores = model.fit_predict(X)

    pivot["is_abnormal_month"] = scores == -1

    return pivot, model, scaler


In [15]:
def explain_monthly_anomalies(profile: pd.DataFrame) -> list[str]:
    advice = []

    normal = profile[profile["is_abnormal_month"] == False]
    abnormal = profile[profile["is_abnormal_month"] == True]

    if len(abnormal) == 0:
        advice.append("Ваш стиль расходов стабилен — резких сбоев не обнаружено.")
        return advice

    baseline = normal.mean()

    for month, row in abnormal.iterrows():
        diff = row.drop("is_abnormal_month") - baseline

        top = diff.sort_values(ascending=False).head(3)

        for cat, value in top.items():
            if value > 0:
                advice.append(
                    f"В {month} траты по категории '{cat}' были выше нормы на {value:.0f} ₽. "
                    "Это ключевая точка для оптимизации."
                )

    return advice


# Категории как точки в смысловом пространстве

In [16]:
CATEGORY_PROTOTYPES = {
    "Еда": "продукты, рестораны, доставка еды, кафе, супермаркеты",
    "Такси": "такси, поездки, транспорт, поездка на машине",
    "Подписки": "подписки, онлайн сервисы, стриминг, премиум доступ",
    "Связь": "мобильная связь, интернет, телефон",
    "Маркетплейсы": "онлайн покупки, интернет магазины",
    "Техника": "компьютеры, серверы, электроника, гаджеты"
}

In [17]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")


def embed(texts):
    return model.encode(texts, normalize_embeddings=True)


def build_category_vectors():
    labels = list(CATEGORY_PROTOTYPES.keys())
    texts = list(CATEGORY_PROTOTYPES.values())
    vecs = embed(texts)

    return labels, vecs


def semantic_classify(df):
    labels, cat_vecs = build_category_vectors()

    desc_vecs = embed(df["description"].astype(str).tolist())

    sims = desc_vecs @ cat_vecs.T  # cosine similarity

    best = sims.argmax(axis=1)

    df["semantic_category"] = [labels[i] for i in best]
    df["semantic_score"] = sims.max(axis=1)

    return df


FINANCE_KEYWORDS = [
    "сбп", "перевод", "перечисление", "копилка",
    "пополнение", "погашение", "внутрибанков",
    "между счетами", "перевод средств"
]


def is_money_movement(desc: str) -> bool:
    desc = desc.lower()
    return any(k in desc for k in FINANCE_KEYWORDS)


def smart_category(df):
    df = df.copy()

    df["is_money"] = df["description"].astype(str).apply(is_money_movement)

    df["mcc_category"] = df["mcc"].apply(classify_by_mcc)
    df = semantic_classify(df)

    df["final_category"] = np.where(
        df["is_money"],
        "Финансовые операции",  # ← выше всего
        np.where(
            df["category"] != "Прочие операции",
            df["category"],  # ← доверяем банку
            np.where(
                df["mcc_category"].notna(),
                df["mcc_category"],
                df["semantic_category"]
            )
        )
    )

    return df



C:\ITSoft\Python\Python3.12.10\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Итоговый Pipeline

In [18]:
def format_user_report(
        df: pd.DataFrame,
        recurring_groups: pd.DataFrame,
        anomalies: pd.DataFrame,
        savings: float,
        profile_advice: List[str]
) -> str:
    text = []

    # ----------------------------
    # 1. Куда уходят деньги
    # ----------------------------
    text.append("📊 КУДА УХОДЯТ ДЕНЬГИ\n")

    by_cat = (
        df.groupby("final_category")["amount"]
        .sum()
        .abs()
        .sort_values(ascending=False)
    )

    total = by_cat.sum()

    for cat, value in by_cat.items():
        share = value / total * 100
        text.append(f"- {cat}: {value:,.0f} ₽ ({share:.1f}%)")

    # ----------------------------
    # 2. Регулярные траты
    # ----------------------------
    text.append("\n🔁 ВАШИ РЕГУЛЯРНЫЕ ПЛАТЕЖИ\n")

    if len(recurring_groups) == 0:
        text.append("Регулярных платежей не найдено.")
    else:
        for _, row in recurring_groups.sort_values("total").iterrows():
            avg = abs(row["total"]) / row["count"]
            text.append(
                f"- {row['description']} → {row['count']} раз, "
                f"≈ {avg:.0f} ₽, всего {abs(row['total']):,.0f} ₽"
            )

    # ----------------------------
    # 3. Аномальные операции
    # ----------------------------
    text.append("\n⚠️ НЕОБЫЧНЫЕ ТРАТЫ\n")

    if len(anomalies) == 0:
        text.append("Аномальных операций не обнаружено.")
    else:
        for _, row in anomalies.sort_values("amount").head(10).iterrows():
            text.append(
                f"- {row['date'].date()} | {row['description'][:50]}… → {row['amount']} ₽"
            )

    # ----------------------------
    # 4. Поведенческий анализ (ML)
    # ----------------------------
    text.append("\n🧠 АНАЛИЗ ВАШЕГО ФИНАНСОВОГО ПОВЕДЕНИЯ\n")

    for line in profile_advice:
        text.append(f"- {line}")

    # ----------------------------
    # 5. Итог по экономии
    # ----------------------------
    text.append("\n💰 ПОТЕНЦИАЛ ЭКОНОМИИ\n")
    text.append(f"Если оптимизировать выявленные привычки, можно сохранить около {abs(savings):,.0f} ₽ за этот период.")

    return "\n".join(text)


In [19]:
def run_pipeline(path: str):
    df = load_bank_xlsx(path)
    df = normalize_columns(df)

    # 1. Реальные категории
    df = smart_category(df)

    # 2. ML для "Прочих операций"
    model, vec = train_category_model(df)
    df = classify_other_operations(df, model, vec)

    # 3. Регулярные платежи
    recurring_groups = detect_recurring_payments(df)

    # 4. Аномалии транзакций
    df = detect_anomalies(df)
    anomalies = df[df["anomaly"] == 1]

    # 5. Поведенческая модель
    profile, _, _ = build_user_profile(df)
    profile_advice = explain_monthly_anomalies(profile)

    # 6. Экономия
    savings = estimate_savings(df, recurring_groups, profile)

    report = format_user_report(
        df,
        recurring_groups,
        anomalies,
        savings,
        profile_advice
    )

    return {
        "report": report,
        "recurring_groups": recurring_groups,
        "anomalies": anomalies,
        "profile": profile,
        "profile_advice": profile_advice,
        "savings": savings,
        "df": df
    }


In [20]:
res = run_pipeline('assets/12 июля 2025 - 11 января 2026.xlsx')
# res['recurring_groups']


In [21]:
res['recurring_groups']

,merchant_id,description,dates,amounts,count,total,cluster,features,is_recurring
13,операция по карте дата создания транзакции мес...,"Операция по карте: 220015******4249, дата созд...","[2025-07-15 00:00:00, 2025-09-03 00:00:00, 202...","[-193.0, -453.0, -688.0]",3,-1334.0,0,"[52.0, 2.0, -444.6666666666667, 202.1687963614...",True
18,операция по карте дата создания транзакции мес...,"Операция по карте: 220015******4249, дата созд...","[2025-08-04 00:00:00, 2025-09-04 00:00:00, 202...","[-199.0, -199.0, -199.0, -199.0, -199.0, -199.0]",6,-1194.0,2,"[31.0, 0.0, -199.0, 0.0]",True
24,операция по карте дата создания транзакции мес...,"Операция по карте: 220015******4249, дата созд...","[2025-09-04 00:00:00, 2025-09-20 00:00:00, 202...","[-64.0, -64.0, -64.0, -64.0]",4,-256.0,2,"[23.333333333333332, 7.1336448530109, -64.0, 0.0]",True
31,операция по карте дата создания транзакции мес...,"Операция по карте: 220015******4249, дата созд...","[2025-10-21 00:00:00, 2025-12-05 00:00:00, 202...","[-755.0, -346.0, -692.0, -346.0, -378.0, -483....",13,-5052.0,1,"[6.75, 12.282270419864018, -388.61538461538464...",True
33,операция по карте дата создания транзакции мес...,"Операция по карте: 220015******4249, дата созд...","[2025-07-16 00:00:00, 2025-10-10 00:00:00, 202...","[-250.0, -250.0, -500.0, -500.0, -500.0]",5,-2000.0,5,"[36.0, 30.797727188868986, -400.0, 122.4744871...",True
34,операция по карте дата создания транзакции мес...,"Операция по карте: 220015******4249, дата созд...","[2025-07-14 00:00:00, 2025-08-19 00:00:00, 202...","[-900.0, -900.0, -900.0, -900.0, -900.0]",5,-4500.0,6,"[37.25, 1.920286436967152, -900.0, 0.0]",True
35,операция по карте дата создания транзакции мес...,"Операция по карте: 220015******4249, дата созд...","[2025-09-21 00:00:00, 2025-10-19 00:00:00, 202...","[-74.0, -64.0, -64.0, -64.0, -83.0]",5,-349.0,7,"[26.0, 28.635642126552707, -69.8, 7.6524505878...",True
39,операция по карте дата создания транзакции мес...,"Операция по карте: 220015******4249, дата созд...","[2025-10-10 00:00:00, 2025-12-08 00:00:00, 202...","[-390.0, -390.0, -390.0]",3,-1170.0,9,"[46.5, 12.5, -390.0, 0.0]",True
40,операция по карте дата создания транзакции мес...,"Операция по карте: 220015******4249, дата созд...","[2025-08-10 00:00:00, 2025-09-11 00:00:00, 202...","[-49.0, -49.0, -49.0, -49.0, -49.0, -49.0]",6,-294.0,2,"[30.6, 0.7999999999999999, -49.0, 0.0]",True
45,операция по карте дата создания транзакции мес...,"Операция по карте: 220015******4249, дата созд...","[2025-10-23 00:00:00, 2025-12-06 00:00:00, 202...","[-458.0, -639.0, -683.0, -765.0, -970.0, -722....",11,-7453.0,11,"[7.6, 13.070577645995604, -677.5454545454545, ...",True


In [22]:
print((res['report']))

📊 КУДА УХОДЯТ ДЕНЬГИ

- Финансовые операции: 132,536 ₽ (67.2%)
- Такси и каршеринг: 32,336 ₽ (16.4%)
- Телефон, интернет, ТВ: 8,206 ₽ (4.2%)
- Фастфуд: 6,316 ₽ (3.2%)
- Транспорт: 5,157 ₽ (2.6%)
- Подписки: 3,294 ₽ (1.7%)
- Аптеки: 2,061 ₽ (1.0%)
- Мультимедиа: 1,823 ₽ (0.9%)
- Кафе и рестораны: 1,690 ₽ (0.9%)
- Путешествия: 1,400 ₽ (0.7%)
- Маркетплейсы: 1,197 ₽ (0.6%)
- Животные: 615 ₽ (0.3%)
- Товары для дома: 327 ₽ (0.2%)
- Прочие расходы: 320 ₽ (0.2%)
- Образование: 15 ₽ (0.0%)

🔁 ВАШИ РЕГУЛЯРНЫЕ ПЛАТЕЖИ

- Операция по карте: 220015******4249, дата создания транзакци → 11 раз, ≈ 678 ₽, всего 7,453 ₽
- Операция по карте: 220015******4249, дата создания транзакци → 13 раз, ≈ 389 ₽, всего 5,052 ₽
- Операция по карте: 220015******4249, дата создания транзакци → 5 раз, ≈ 900 ₽, всего 4,500 ₽
- Подписка "Альфа-Смарт" за период с 09.08.25 по 08.09.25 → 4 раз, ≈ 599 ₽, всего 2,396 ₽
- Операция по карте: 220015******4249, дата создания транзакци → 5 раз, ≈ 400 ₽, всего 2,000 ₽
- Операция п